## 构建问题提示语

### 没有提示语模板时

应当直接将 prompt 作为第一条 user 消息：

In [18]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM()
llm("写一首诗词吧", verbose=True, new_chat=True)
llm.memory

[INFO] 记住 10 轮对话
[INFO] FakeLLM: [{'role': 'user', 'content': '写一首诗词吧'}]
这是一个模拟调用!

[{'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 有提示语模板，但未使用 task 变量

应当在 system 后，将 prompt 作为 user 消息：

In [14]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[
        ("system", Template(text="请你帮我写首歌"))
    ]
)
llm("还是写一首诗词吧", verbose=True, new_chat=True)
llm.memory

[INFO] 记住 10 轮对话
[INFO] FakeLLM: [{'role': 'system', 'content': '请你帮我写首歌'}]
这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写首歌'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [15]:
llm.provider_dict

{'last_input': '还是写一首诗词吧', 'last_output': '这是一个模拟调用!', 'task': '还是写一首诗词吧'}

### 使用 task 变量，且列表最后一条是 user

In [6]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[
        ("system", Template(text="请你帮我写{{task}}"))
    ]
)
llm("一首诗词", verbose=True, new_chat=True)
llm.memory

[INFO] 记住 10 轮对话
[INFO] FakeLLM: [{'role': 'system', 'content': '请你帮我写一首诗词'}]
这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写一首诗词'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 使用 task 变量，且列表最后一条不是 user

In [3]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[
        ("system", Template(text="请你帮我写{{task}}")),
        ("user", "请开始")
    ]
)
llm("一首诗词", verbose=True, new_chat=True)
llm.memory

[INFO] 记住 10 轮对话
[INFO] FakeLLM: [{'role': 'system', 'content': '请你帮我写一首诗词'}, {'role': 'user', 'content': '请开始'}]
这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写一首诗词'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

## 映射提示语中的模板变量

### 默认映射

In [5]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[Template(text="请你帮我写{{task}}"), "请开始"]
)
llm("一首儿歌，4行")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 重新映射

In [3]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[Template(text="请你帮我写{{mytask}}"), "请开始"],
    template_binding={
        "mytask": "task"
    }
)
llm("一首儿歌，4行")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

## 携带背景知识

### 文本知识

### 动态查询

## 管理记忆长度

### 默认记住10轮

In [3]:
a("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

[INFO] 仅记住 10 轮对话
小兔子乖乖，梦中乐逍遥，

胡萝卜成山，幸福在身旁。 

（注：这是一首简短的儿歌，希望可以满足您的需求）
[USAGE] {"input_tokens": 21, "output_tokens": 36, "total_tokens": 57}


'小兔子乖乖，梦中乐逍遥，\n\n胡萝卜成山，幸福在身旁。 \n\n（注：这是一首简短的儿歌，希望可以满足您的需求）'

In [3]:
await a("你能帮我写一首关于兔子做梦的四句儿歌?", handlers=[alog])

小兔子乖乖，梦中舞起来，

胡萝卜做山，月宫乐翻天。 

（注：这是一个创作的简单例子，希望能符合您的期望）


'小兔子乖乖，梦中舞起来，\n\n胡萝卜做山，月宫乐翻天。 \n\n（注：这是一个创作的简单例子，希望能符合您的期望）'

In [5]:
a.get_chat_memory(2)

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小兔子乖乖躺床上，\n\n梦中萝卜堆成山，\n\n月光轻轻照进窗，\n\n幸福入眠不用赶。'}]

In [5]:
a("换成小羊吧")

小羊儿乖乖，梦里跳啊跳，
青草做地毯，云朵笑弯腰。


'小羊儿乖乖，梦里跳啊跳，\n青草做地毯，云朵笑弯腰。'

### 限制记忆中的对话轮数

In [6]:
a("改为在蓝天上")

小羊儿乖乖，梦里跳啊跳，
青草铺蓝天，白云身旁绕。


'小羊儿乖乖，梦里跳啊跳，\n青草铺蓝天，白云身旁绕。'

In [9]:
a.get_chat_memory(1)

[{'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': '小羊儿乖乖，梦里跳啊跳，\n青草铺蓝天，白云身旁绕。'}]

In [4]:
a("改为在水塘", remember_rounds=1, verbose=True)

[INFO] 仅记住 1 轮对话
好的，可以这样修改：

小兔子乖乖，梦中乐逍遥，
水塘清如镜，快乐水中跳。

这样可以吗？这首儿歌描绘了小兔子在梦中于清澈的水塘边欢乐跳跃的场景。
[USAGE] {"input_tokens": 71, "output_tokens": 50, "total_tokens": 121}


'好的，可以这样修改：\n\n小兔子乖乖，梦中乐逍遥，\n水塘清如镜，快乐水中跳。\n\n这样可以吗？这首儿歌描绘了小兔子在梦中于清澈的水塘边欢乐跳跃的场景。'

In [11]:
a.get_chat_memory(1)

[{'role': 'user', 'content': '改为在水塘'},
 {'role': 'assistant',
  'content': '小羊儿乖乖，梦里跳啊跳，\n青草绕水塘，倒映白云朵。 \n\n如果需要调整或有其他要求，请告诉我！我可以根据您的需求再做修改。'}]